In [1]:
import pandas as pd
import glob
import os

In [32]:
# Cartella con i CSV annuali filtrati
folder = "./dataR3000/"
files = sorted(glob.glob(os.path.join(folder, "*/filtered_tickers.csv")))  # ordinati per nome

In [33]:
data = []
# Carica tutti i file e aggiungi colonna Year
for f in files:
    year = 2000 + int( f.split("\\")[1].split("-")[0] )
    df = pd.read_csv(f)
    df["year"] = year
    data.append(df[["ticker", "year"]])

all_data = pd.concat(data, ignore_index=True)

In [34]:
# Trova primo e ultimo anno per ogni ticker
ingressi = all_data.groupby("ticker")["year"].min().rename("ingresso")
uscite = all_data.groupby("ticker")["year"].max().rename("uscita")

# Anno massimo disponibile
last_year = all_data["year"].max() + 1

# Aggiusta l'uscita: +1, tranne se è l'ultimo anno
uscite = uscite.apply(lambda x: 0 if x + 1 == last_year else x + 1)

In [35]:
# Dataframe finale
result = pd.DataFrame({"ticker": all_data["ticker"].unique()})
result = result.merge(ingressi, on="ticker")
result = result.merge(uscite, on="ticker")

In [36]:
# Se il ticker è ancora presente nell'ultimo anno, setta uscita=0
result.loc[result["uscita"] == last_year, "uscita"] = 0

In [37]:
result.to_csv("./historical_total_tickers.csv", index=False)